# Transfer Learning
## 6.1 

### Assignment Solution - Predict Bitcoin Pricing

In [6]:
!pip install tensorflow==2.0.0-beta0

In [0]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np

### Mount the folder from my Drive

In [8]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### Read in the pricing data

In [0]:
df = pd.read_csv('/content/gdrive/My Drive/bitcoin/coinbaseUSD_1-min_data_2014-12-01_to_2019-01-09.csv')

### What is in the data?

In [21]:
df.head()

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
0,1417411980,300.0,300.0,300.0,300.0,0.01,3.0,300.0
1,1417412040,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1417412100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1417412160,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1417412220,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
df.dropna(axis=0, inplace=True) # drop rows with NaN
df.head()

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
0,1417411980,300.0,300.0,300.0,300.0,0.010000,3.00000,300.0
7,1417412400,300.0,300.0,300.0,300.0,0.010000,3.00000,300.0
51,1417415040,370.0,370.0,370.0,370.0,0.010000,3.70000,370.0
77,1417416600,370.0,370.0,370.0,370.0,0.026556,9.82555,370.0
1436,1417498140,377.0,377.0,377.0,377.0,0.010000,3.77000,377.0


In [0]:
df['datetime'] = pd.to_datetime(df['Timestamp'],unit='s')

In [24]:
df.head()

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,datetime
0,1417411980,300.0,300.0,300.0,300.0,0.010000,3.00000,300.0,2014-12-01 05:33:00
7,1417412400,300.0,300.0,300.0,300.0,0.010000,3.00000,300.0,2014-12-01 05:40:00
51,1417415040,370.0,370.0,370.0,370.0,0.010000,3.70000,370.0,2014-12-01 06:24:00
77,1417416600,370.0,370.0,370.0,370.0,0.026556,9.82555,370.0,2014-12-01 06:50:00
1436,1417498140,377.0,377.0,377.0,377.0,0.010000,3.77000,377.0,2014-12-02 05:29:00


In [25]:
df.tail()

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,datetime
2099755,1546898520,4006.01,4006.57,4006.00,4006.01,3.382954,13553.433078,4006.390309,2019-01-07 22:02:00
2099756,1546898580,4006.01,4006.57,4006.00,4006.01,0.902164,3614.083169,4006.017233,2019-01-07 22:03:00
2099757,1546898640,4006.01,4006.01,4006.00,4006.01,1.192123,4775.647308,4006.003635,2019-01-07 22:04:00
2099758,1546898700,4006.01,4006.01,4005.50,4005.50,2.699700,10814.241898,4005.719991,2019-01-07 22:05:00
2099759,1546898760,4005.51,4006.01,4005.51,4005.99,1.752778,7021.183546,4005.745614,2019-01-07 22:06:00


### set an index

In [0]:
df = df.set_index(['datetime'])

In [27]:
df.head()

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
datetime,,,,,,,,
2014-12-01 05:33:00,1417411980,300.0,300.0,300.0,300.0,0.010000,3.00000,300.0
2014-12-01 05:40:00,1417412400,300.0,300.0,300.0,300.0,0.010000,3.00000,300.0
2014-12-01 06:24:00,1417415040,370.0,370.0,370.0,370.0,0.010000,3.70000,370.0
2014-12-01 06:50:00,1417416600,370.0,370.0,370.0,370.0,0.026556,9.82555,370.0
2014-12-02 05:29:00,1417498140,377.0,377.0,377.0,377.0,0.010000,3.77000,377.0


### resample to mean daily values

In [28]:
mean_daily_values = df.resample('D').mean()
mean_daily_values.head()

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
datetime,,,,,,,,
2014-12-01,1.417414e+09,335.000000,335.000000,335.000000,335.000000,0.014139,4.881387,335.000000
2014-12-02,1.417528e+09,377.821429,377.857143,377.821429,377.857143,2.144800,810.724043,377.854911
2014-12-03,1.417612e+09,377.818333,377.818333,377.818333,377.818333,0.091101,34.419664,377.818333
2014-12-04,1.417658e+09,377.100000,377.100000,377.100000,377.100000,0.010000,3.771000,377.100000
2014-12-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
mean_daily_values.drop(['Timestamp'], axis=1, inplace=True)
mean_daily_values.head()

,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
datetime,,,,,,,
2014-12-01,335.000000,335.000000,335.000000,335.000000,0.014139,4.881387,335.000000
2014-12-02,377.821429,377.857143,377.821429,377.857143,2.144800,810.724043,377.854911
2014-12-03,377.818333,377.818333,377.818333,377.818333,0.091101,34.419664,377.818333
2014-12-04,377.100000,377.100000,377.100000,377.100000,0.010000,3.771000,377.100000
2014-12-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### drop rows with missing values

In [31]:
mean_daily_values.dropna(axis=0, inplace=True)
mean_daily_values.head()

,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
datetime,,,,,,,
2014-12-01,335.000000,335.000000,335.000000,335.000000,0.014139,4.881387,335.000000
2014-12-02,377.821429,377.857143,377.821429,377.857143,2.144800,810.724043,377.854911
2014-12-03,377.818333,377.818333,377.818333,377.818333,0.091101,34.419664,377.818333
2014-12-04,377.100000,377.100000,377.100000,377.100000,0.010000,3.771000,377.100000
2014-12-06,378.000000,378.000000,378.000000,378.000000,0.015000,5.670000,378.000000


In [32]:
mean_daily_values.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1465 entries, 2014-12-01 to 2019-01-07
Data columns (total 7 columns):
Open                 1465 non-null float64
High                 1465 non-null float64
Low                  1465 non-null float64
Close                1465 non-null float64
Volume_(BTC)         1465 non-null float64
Volume_(Currency)    1465 non-null float64
Weighted_Price       1465 non-null float64
dtypes: float64(7)
memory usage: 91.6 KB


### split data into training and validation/test

In [0]:
test_data = mean_daily_values['2019']
train_data = mean_daily_values['2014':'2018']

### Reset indices to a time step

In [34]:
test_data.reset_index(inplace=True)
train_data.reset_index(inplace=True)
test_data.head()

,datetime,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
0,2019-01-01,3700.554931,3701.321181,3699.788875,3700.670507,7.185827,26669.369776,3700.539528
1,2019-01-02,3823.154149,3824.123544,3822.187012,3823.210257,6.679140,25633.288929,3823.152158
2,2019-01-03,3829.991333,3830.601222,3829.190292,3829.922118,6.139066,23488.491210,3829.900978
3,2019-01-04,3785.379042,3786.126722,3784.568264,3785.405590,6.215354,23523.548116,3785.337456
4,2019-01-05,3834.627896,3835.344944,3833.723542,3834.609424,4.206899,16131.900905,3834.565401


### do it again

In [36]:
test_data.reset_index(inplace=True)
train_data.reset_index(inplace=True)
test_data.head()

,index,datetime,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
0,0,2019-01-01,3700.554931,3701.321181,3699.788875,3700.670507,7.185827,26669.369776,3700.539528
1,1,2019-01-02,3823.154149,3824.123544,3822.187012,3823.210257,6.679140,25633.288929,3823.152158
2,2,2019-01-03,3829.991333,3830.601222,3829.190292,3829.922118,6.139066,23488.491210,3829.900978
3,3,2019-01-04,3785.379042,3786.126722,3784.568264,3785.405590,6.215354,23523.548116,3785.337456
4,4,2019-01-05,3834.627896,3835.344944,3833.723542,3834.609424,4.206899,16131.900905,3834.565401


### Extract time steps

In [0]:
test_y = test_data[['index']].to_numpy()
test_x = test_data[['Close']].to_numpy()

train_y = test_data[['index']].to_numpy()
train_x = test_data[['Close']].to_numpy()

### create a generator

In [0]:
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

N_INPUT = 5
BATCH_SIZE = 16

train_gen = TimeseriesGenerator(train_x, train_y, length=N_INPUT, sampling_rate=7, batch_size=BATCH_SIZE)
test_gen = TimeseriesGenerator(test_x, test_y, length=N_INPUT, sampling_rate=7, batch_size=BATCH_SIZE)

### create the model

In [44]:
batch_0 = train_gen[0]
x, y = batch_0

model = keras.Sequential([
    keras.layers.Bidirectional(keras.layers.LSTM(100, recurrent_dropout=0.5), input_shape=(x.shape[1], x.shape[2])),
    keras.layers.Dense(1)
])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (None, 200)               81600     
_________________________________________________________________
dense (Dense)                (None, 1)                 201       
Total params: 81,801
Trainable params: 81,801
Non-trainable params: 0
_________________________________________________________________
None


In [45]:
model.compile(optimizer='adam', loss='mse')
model.fit_generator(train_gen, epochs=10)

Epoch 1/10
1/1 [==============================] - 2s 2s/step - loss: 25.6449
Epoch 2/10
1/1 [==============================] - 0s 8ms/step - loss: 25.1330
Epoch 3/10
1/1 [==============================] - 0s 80ms/step - loss: 24.7024
Epoch 4/10
1/1 [==============================] - 0s 104ms/step - loss: 24.2509
Epoch 5/10
1/1 [==============================] - 0s 95ms/step - loss: 23.5719
Epoch 6/10
1/1 [==============================] - 0s 99ms/step - loss: 22.0631
Epoch 7/10
1/1 [==============================] - 0s 99ms/step - loss: 20.6728
Epoch 8/10
1/1 [==============================] - 0s 100ms/step - loss: 19.9951
Epoch 9/10
1/1 [==============================] - 0s 95ms/step - loss: 19.4833
Epoch 10/10
1/1 [==============================] - 0s 101ms/step - loss: 19.1002
